# Part 1 : Web Scraping

#### Import The data from Wikepedia

In [136]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = pd.read_html(str(table))
dataset = df[0]
headers = dataset.iloc[0]
dataset  = pd.DataFrame(dataset.values[1:], columns=headers)
dataset.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Create the dataframe from the html

In [137]:
unique_Postcode = dataset["Postcode"].unique()
datasetUniquePost = pd.DataFrame([], columns=dataset.columns.values )

for i, val in enumerate(unique_Postcode): 
    f1 = dataset[dataset['Postcode']==val]
    if len(f1) > 1 :
        newN = ""
        i=0;
        for index, row in f1.iterrows():
            if i < (len(f1)-1) : 
                newN = newN + row['Neighbourhood'] + ", " 
                i = i+1
            else :
                newN = newN + row['Neighbourhood'] 
                i = i+1
        df2 = pd.DataFrame({"Postcode":row['Postcode'],"Borough" :  row['Borough'],"Neighbourhood":newN },index=[0]) 
        datasetUniquePost = df_temp.append(df2)
    else : 
        geoloc = df_temp.append(f1) 

#### if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [138]:
withNoNeighbourhoodIndex  = datasetUniquePost[datasetUniquePost["Neighbourhood"]=="Not assigned"].index
datasetUniquePost.loc[withNoNeighbourhoodIndex,'Neighbourhood'] = datasetUniquePost.loc[withNoNeighbourhoodIndex,'Borough']
datasetUniquePost.shape

# Part 2

In [140]:
geoloc = pd.read_csv("Geospatial_Coordinates.csv")
geoloc.rename(columns={"Postal Code": "Postcode"},inplace=True)
datasetUniquePost = datasetUniquePost.join(geoloc.set_index('Postcode'), on='Postcode')
datasetUniquePost.reset_index(drop=True,inplace=True)
datasetUniquePost.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [141]:
datasetUniquePost.shape

(104, 5)